<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [5]:
print(yelp.shape)

yelp.head()

(10000, 9)


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [24]:
# using spacy and the tokenizer
# maybe will have to switch to md or sm if it causes my comp problems
import spacy
from spacy.tokenizer import Tokenizer

In [23]:
# load the model
nlp = spacy.load('en_core_web_lg')

In [25]:
# instantiate the tokenizer 
tokenizer = Tokenizer(nlp.vocab)

In [26]:
# f(x) to tokenize a single doc
def tokenize(doc):
    return [token.text for token in tokenizer(doc)]

In [27]:
# Column of tokenized text
yelp['tokens'] = [tokenize(doc) for doc in yelp['text'].values]

In [28]:
# and it seems to have worked
yelp['tokens'].head(10)

0    [BEWARE!!!, FAKE,, FAKE,, FAKE....We, also, ow...
1    [Came, here, for, lunch, Togo., Service, was, ...
2    [I've, been, to, Vegas, dozens, of, times, and...
3    [We, went, here, on, a, night, where, they, cl...
4    [3.5, to, 4, stars, \n\n, Not, bad, for, the, ...
5    [Tasty,, fast, casual, Latin, street, food.,  ...
6    [This, show, is, absolutely, amazing!!, What, ...
7    [Came, for, the, Pho, and, really, enjoyed, it...
8    [Absolutely, the, most, Unique, experience, in...
9    [Wow., I, walked, in, and, sat, at, the, bar, ...
Name: tokens, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [30]:
# join back the token lists into strings
data =  [' '.join(s) for s in yelp['tokens']]

len(data)

10000

In [31]:
# using the standard english stop words, and cutting out 5% outliers
vect = TfidfVectorizer(stop_words='english', min_df=0.05, max_df=0.95)

# fit to the data
vect.fit(data)

# vectorize data into sparse matrix
sparse = vect.transform(data)

# create a df of the matrix but filled in with zeros
dtm = pd.DataFrame(sparse.todense(), columns=vect.get_feature_names())

dtm.head()

,10,amazing,area,asked,away,awesome,bad,bar,best,better,...,vegas,wait,want,wanted,wasn,way,went,work,worth,years
0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,...,0.000000,0.392384,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.14922,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,...,0.292434,0.000000,0.0,0.0,0.0,0.0,0.123145,0.0,0.0,0.0
3,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.000000,0.360664,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.381265,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.0,0.0,0.23316,0.466138,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [32]:
# instantiate the model very simply
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

# fit to dataframe from vectorized data
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [33]:
# fake review to test model
fake_review = ['''
    This product is terrible. It broke the first time I used it.
    I want my money back
''']

In [34]:
# vectorize review
search = vect.transform(fake_review)

# run the search
nn.kneighbors(search.todense())

(array([[0.        , 0.60053305, 0.60053305, 0.60053305, 0.82027879]]),
 array([[9037, 9181, 5547, 8411, 5518]]))

### Model not great but these reviews all do have strongly negative sentiments so it nailed that.

In [35]:
data[9037]

'If you want to be seen by a DR. who looks at his laptop more than he looks at you or spend time with you or even listen to you, this is the one to go to!'

In [36]:
data[8411]

"The man that answers the phone has a horrible attitude. If they do NOT want to do Delivary shouldn't be advertising as so."

In [37]:
data[5547]

'Over priced hipster jive ass baking. The chocolate babka is getting them the extra star but you can pass on the rest unless you want some greasy croissants or some other stomach ache inducing nonsense. Keep on walking. Nothing to see here.'

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [40]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD

In [43]:
svd = TruncatedSVD(n_iter=7)
vect = TfidfVectorizer(tokenizer=tokenize)
clf = RandomForestClassifier()

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

In [44]:
parameters = {
    'lsi__vect__max_df': [0.925, 0.95, 1.0],
    'lsi__vect__ngram_range': [(1,1), (1,2)],
    'lsi__svd__n_components': [2,5,10],
    'clf__n_estimators': [50, 100, 200],
    'clf__max_depth': [3,4,5]
}

In [48]:
X = yelp['text']
y = yelp['stars']

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=4, verbose=10)
grid_search.fit(X, y)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Batch computation too fast (0.1727s.) Setting batch_size=2.
[Parallel(n_jobs=4)]: Batch computation too fast (0.0090s.) Setting batch_size=88.


PicklingError: Could not pickle the task to send it to the workers.

In [39]:
yelp['stars']

0       1
1       4
2       3
3       1
4       4
5       4
6       5
7       4
8       5
9       1
10      4
11      1
12      5
13      4
14      1
15      4
16      5
17      5
18      1
19      5
20      5
21      5
22      2
23      4
24      2
25      5
26      4
27      5
28      5
29      5
       ..
9970    5
9971    5
9972    5
9973    2
9974    5
9975    5
9976    5
9977    1
9978    5
9979    1
9980    1
9981    5
9982    5
9983    5
9984    5
9985    5
9986    1
9987    5
9988    5
9989    5
9990    5
9991    5
9992    5
9993    1
9994    4
9995    1
9996    3
9997    2
9998    4
9999    1
Name: stars, Length: 10000, dtype: int64

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [3]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [1]:
id2word = ...

Create a bag of words representation of the entire corpus

In [ ]:
corpus = ...

Your LDA model should be ready for estimation: 

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)